In [1]:
import json
import os
import math
import pandas as pd
import pickle
import re
import requests
from unidecode import unidecode
from scrape_cardlists import *

## load commanders

In [2]:
# load data
with open(os.path.abspath(os.path.join('..','data','cards.pkl')), 'rb') as f:
    cards = pickle.load(f)
with open(os.path.abspath(os.path.join('..','data','commanders.pkl')), 'rb') as f:
    commanders = pickle.load(f)
with open(os.path.abspath(os.path.join('..','data','edhreclists.pkl')), 'rb') as f:
    comm_lists = pickle.load(f)



## data exploration

In [5]:
cards.columns

Index(['artist', 'artistIds', 'asciiName', 'attractionLights', 'availability',
       'boosterTypes', 'borderColor', 'cardParts', 'colorIdentity',
       'colorIndicator', 'colors', 'defense', 'duelDeck', 'edhrecRank',
       'edhrecSaltiness', 'faceConvertedManaCost', 'faceFlavorName',
       'faceManaValue', 'faceName', 'finishes', 'flavorName', 'flavorText',
       'frameEffects', 'frameVersion', 'hand', 'hasAlternativeDeckLimit',
       'hasContentWarning', 'hasFoil', 'hasNonFoil', 'isAlternative',
       'isFullArt', 'isFunny', 'isOnlineOnly', 'isOversized', 'isPromo',
       'isRebalanced', 'isReprint', 'isReserved', 'isStarter',
       'isStorySpotlight', 'isTextless', 'isTimeshifted', 'keywords',
       'language', 'layout', 'leadershipSkills', 'life', 'loyalty', 'manaCost',
       'manaValue', 'name', 'number', 'originalPrintings',
       'originalReleaseDate', 'originalText', 'originalType', 'otherFaceIds',
       'power', 'printings', 'promoTypes', 'rarity', 'rebalancedPrint

In [14]:
fail = []
for card in commanders['valid']:
    if card in cards['name']:
        continue
    fail.append(card)
len(fail)

200

In [16]:
commanders['valid'][0]

'Stangg, Echo Warrior'

In [29]:
stangg = cards[cards['name'].str.contains('Stangg, Echo Warrior')]
for col in stangg.columns:
    print(f'{col}: {len(stangg[col].unique())} unique values: {stangg[col].unique()}')

artist: 1 unique values: ['Randy Vargas']
artistIds: 1 unique values: ['d20672ca-0555-4238-a984-fd171d36b247']
asciiName: 1 unique values: [nan]
attractionLights: 1 unique values: [nan]
availability: 2 unique values: ['mtgo, paper' 'paper']
boosterTypes: 1 unique values: [nan]
borderColor: 1 unique values: ['black']
cardParts: 1 unique values: [nan]
colorIdentity: 1 unique values: ['G, R']
colorIndicator: 1 unique values: [nan]
colors: 1 unique values: ['G, R']
defense: 1 unique values: [nan]
duelDeck: 1 unique values: [nan]
edhrecRank: 1 unique values: [10905.]
edhrecSaltiness: 1 unique values: [nan]
faceConvertedManaCost: 1 unique values: [nan]
faceFlavorName: 1 unique values: [nan]
faceManaValue: 1 unique values: [nan]
faceName: 1 unique values: [nan]
finishes: 2 unique values: ['nonfoil, foil' 'etched']
flavorName: 1 unique values: [nan]
flavorText: 1 unique values: [nan]
frameEffects: 2 unique values: ['legendary' 'legendary, inverted']
frameVersion: 1 unique values: [2015]
hand: 

In [20]:
cards.shape

(87566, 78)

In [60]:
unique = cards.drop_duplicates(subset=['name', 'colorIdentity', 'text'])
with open(os.path.abspath(os.path.join('..','data','cards_unique.pkl')), 'wb') as f:
    pickle.dump(unique, f)

In [35]:
duptext = []
dupcolor = []
for name, group in cards.groupby('name'):
    if len(group['colorIdentity'].unique()) > 1:
        dupcolor.append(name)
    if len(group['text'].unique()) > 1:
        duptext.append(name)

In [53]:
len(dupcolor), len(duptext)

(1, 670)

In [47]:
duptext

['A-Alrund, God of the Cosmos // A-Hakka, Whispering Raven',
 'A-Binding Geist // A-Spectral Binding',
 "A-Blessed Hippogriff // A-Tyr's Blessing",
 'A-Brine Comber // A-Brinebound Gift',
 'A-Devoted Grafkeeper // A-Departed Soulkeeper',
 "A-Dorothea, Vengeful Victim // A-Dorothea's Retribution",
 'A-Emerald Dragon // A-Dissonant Wave',
 'A-Gutter Skulker // A-Gutter Shortcut',
 "A-Lantern Bearer // A-Lanterns' Lift",
 'A-Mischievous Catgeist // A-Catlike Curiosity',
 'A-Rowan, Scholar of Sparks // A-Will, Scholar of Frost',
 'A-Young Blue Dragon // A-Sand Augury',
 'A-Young Red Dragon // A-Bathe in Gold',
 'Aberrant Researcher // Perfected Form',
 'Accident-Prone Apprentice // Amphibian Accident',
 'Accursed Witch // Infectious Curse',
 'Aclazotz, Deepest Betrayal // Temple of the Dead',
 'Aetherblade Agent // Gitaxian Mindstinger',
 'Afflicted Deserter // Werewolf Ransacker',
 "Agadeem's Awakening // Agadeem, the Undercrypt",
 'Akki Lavarunner // Tok-Tok, Volcano Born',
 'Akoum Warri

In [48]:
cards.columns

Index(['artist', 'artistIds', 'asciiName', 'attractionLights', 'availability',
       'boosterTypes', 'borderColor', 'cardParts', 'colorIdentity',
       'colorIndicator', 'colors', 'defense', 'duelDeck', 'edhrecRank',
       'edhrecSaltiness', 'faceConvertedManaCost', 'faceFlavorName',
       'faceManaValue', 'faceName', 'finishes', 'flavorName', 'flavorText',
       'frameEffects', 'frameVersion', 'hand', 'hasAlternativeDeckLimit',
       'hasContentWarning', 'hasFoil', 'hasNonFoil', 'isAlternative',
       'isFullArt', 'isFunny', 'isOnlineOnly', 'isOversized', 'isPromo',
       'isRebalanced', 'isReprint', 'isReserved', 'isStarter',
       'isStorySpotlight', 'isTextless', 'isTimeshifted', 'keywords',
       'language', 'layout', 'leadershipSkills', 'life', 'loyalty', 'manaCost',
       'manaValue', 'name', 'number', 'originalPrintings',
       'originalReleaseDate', 'originalText', 'originalType', 'otherFaceIds',
       'power', 'printings', 'promoTypes', 'rarity', 'rebalancedPrint

In [50]:
cards[cards['name'] == 'A-Alrund, God of the Cosmos // A-Hakka, Whispering Raven']['text'].unique()

array(['Alrund gets +1/+1 for each card in your hand and each foretold card you own in exile.\\nAt the beginning of your end step, choose a card type, then reveal the top three cards of your library. Put all cards of the chosen type revealed this way into your hand and the rest on the bottom of your library in any order.',
       "Flying\\nWhenever Hakka, Whispering Raven deals combat damage to a player, return it to its owner's hand, then scry 2."],
      dtype=object)

In [55]:
cards.drop_duplicates(subset=['name', 'text']).shape

(28781, 78)

In [59]:
cards.drop_duplicates(subset=['text']).shape

(27060, 78)

## cardlist stuff

In [61]:
def format_commander_name(commander_name:str):
    non_alphas_regex = "[^\w\s-]" # Remove everything that's not alphanumeric or space or hyphen
    formatted_name = re.sub(non_alphas_regex, "", commander_name)
    formatted_name = formatted_name.lower() # Make lowercase
    formatted_name = formatted_name.replace(" ", "-")  # Replace spaces with hyphens
    formatted_name = unidecode(formatted_name)
    print(f"In format_commander_name and formatted name is {formatted_name}")
    return formatted_name

In [62]:
format_commander_name("Orvar, the All-Form")

In format_commander_name and formatted name is orvar-the-all-form


'orvar-the-all-form'

In [43]:
commander_name = fail[0]

formatted_name = format_commander_name(commander_name)
json_url = f"https://json.edhrec.com/pages/commanders/{formatted_name}.json"
response = requests.get(json_url)
if response.status_code == 200:
    json_data = response.json()
    print(f"JSON request successful!")
    data = json_data
    print(sorted(data['cardlist'], key=lambda card: card['num_decks'], reverse=True))
else:
    print(f"JSON request failed! Try different commander name")


In format_commander_name and formatted name is galadriel-of-lothlórien
JSON request failed! Try different commander name


In [23]:
data['container']

{'description': 'Popular decks and cards for Stangg, Echo Warrior',
 'json_dict': {'cardlists': [{'cardviews': [{'name': "Etali's Favor",
      'sanitized': 'etalis-favor',
      'sanitized_wo': 'etalis-favor',
      'url': '/cards/etalis-favor',
      'synergy': 0.89,
      'inclusion': 149,
      'label': '95% of 157 decks\n+89% synergy',
      'num_decks': 149,
      'potential_decks': 157},
     {'name': 'Diamond Pick-Axe',
      'sanitized': 'diamond-pick-axe',
      'sanitized_wo': 'diamond-pick-axe',
      'url': '/cards/diamond-pick-axe',
      'synergy': 0.15,
      'inclusion': 38,
      'label': '21% of 180 decks\n+15% synergy',
      'num_decks': 38,
      'potential_decks': 180},
     {'name': "Tarrian's Soulcleaver",
      'sanitized': 'tarrians-soulcleaver',
      'sanitized_wo': 'tarrians-soulcleaver',
      'url': '/cards/tarrians-soulcleaver',
      'synergy': 0.08,
      'inclusion': 29,
      'label': '16% of 180 decks\n+8% synergy',
      'num_decks': 29,
      'po

In [34]:
sorted(data['cardlist'], key=lambda card: card['num_decks'], reverse=True)
data['cardlist'][:5]

[{'name': "Etali's Favor",
  'sanitized': 'etalis-favor',
  'sanitized_wo': 'etalis-favor',
  'url': '/cards/etalis-favor',
  'synergy': 0.89,
  'num_decks': 149,
  'potential_decks': 157},
 {'name': 'Command Tower',
  'sanitized': 'command-tower',
  'sanitized_wo': 'command-tower',
  'url': '/cards/command-tower',
  'synergy': 0.06,
  'num_decks': 1405,
  'potential_decks': 1598},
 {'name': 'Chishiro, the Shattered Blade',
  'sanitized': 'chishiro-the-shattered-blade',
  'sanitized_wo': 'chishiro-the-shattered-blade',
  'url': '/cards/chishiro-the-shattered-blade',
  'synergy': 0.81,
  'num_decks': 1347,
  'potential_decks': 1596},
 {'name': 'Sticky Fingers',
  'sanitized': 'sticky-fingers',
  'sanitized_wo': 'sticky-fingers',
  'url': '/cards/sticky-fingers',
  'synergy': 0.78,
  'num_decks': 1331,
  'potential_decks': 1588},
 {'name': 'Sol Ring',
  'sanitized': 'sol-ring',
  'sanitized_wo': 'sol-ring',
  'url': '/cards/sol-ring',
  'synergy': 0.04,
  'num_decks': 1321,
  'potential_

In [35]:
flat = []
for cardlist in data['container']['json_dict']['cardlists']:
    for view in cardlist['cardviews']:
        flat.append(view)
flat[:5]

[{'name': "Etali's Favor",
  'sanitized': 'etalis-favor',
  'sanitized_wo': 'etalis-favor',
  'url': '/cards/etalis-favor',
  'synergy': 0.89,
  'inclusion': 149,
  'label': '95% of 157 decks\n+89% synergy',
  'num_decks': 149,
  'potential_decks': 157},
 {'name': 'Diamond Pick-Axe',
  'sanitized': 'diamond-pick-axe',
  'sanitized_wo': 'diamond-pick-axe',
  'url': '/cards/diamond-pick-axe',
  'synergy': 0.15,
  'inclusion': 38,
  'label': '21% of 180 decks\n+15% synergy',
  'num_decks': 38,
  'potential_decks': 180},
 {'name': "Tarrian's Soulcleaver",
  'sanitized': 'tarrians-soulcleaver',
  'sanitized_wo': 'tarrians-soulcleaver',
  'url': '/cards/tarrians-soulcleaver',
  'synergy': 0.08,
  'inclusion': 29,
  'label': '16% of 180 decks\n+8% synergy',
  'num_decks': 29,
  'potential_decks': 180},
 {'name': 'Roaming Throne',
  'sanitized': 'roaming-throne',
  'sanitized_wo': 'roaming-throne',
  'url': '/cards/roaming-throne',
  'synergy': -0.18,
  'inclusion': 11,
  'label': '7% of 157 d

In [31]:
(data['container']['json_dict']['cardlists'][0]['cardviews'])

[{'name': "Etali's Favor",
  'sanitized': 'etalis-favor',
  'sanitized_wo': 'etalis-favor',
  'url': '/cards/etalis-favor',
  'synergy': 0.89,
  'inclusion': 149,
  'label': '95% of 157 decks\n+89% synergy',
  'num_decks': 149,
  'potential_decks': 157},
 {'name': 'Diamond Pick-Axe',
  'sanitized': 'diamond-pick-axe',
  'sanitized_wo': 'diamond-pick-axe',
  'url': '/cards/diamond-pick-axe',
  'synergy': 0.15,
  'inclusion': 38,
  'label': '21% of 180 decks\n+15% synergy',
  'num_decks': 38,
  'potential_decks': 180},
 {'name': "Tarrian's Soulcleaver",
  'sanitized': 'tarrians-soulcleaver',
  'sanitized_wo': 'tarrians-soulcleaver',
  'url': '/cards/tarrians-soulcleaver',
  'synergy': 0.08,
  'inclusion': 29,
  'label': '16% of 180 decks\n+8% synergy',
  'num_decks': 29,
  'potential_decks': 180},
 {'name': 'Roaming Throne',
  'sanitized': 'roaming-throne',
  'sanitized_wo': 'roaming-throne',
  'url': '/cards/roaming-throne',
  'synergy': -0.18,
  'inclusion': 11,
  'label': '7% of 157 d

## build synergy baseline decks

In [4]:
comm = commanders['valid'][0]
baseline = {}
for split in lists:
    baseline[split] = {}
    for comm in lists[split]:
        baseline[split][comm] = lists[split][comm][:99]

In [32]:
short = []
fail = []
for commander in commanders['test']+commanders['valid']:
    if commander in lists['valid']:
        if len(lists['valid'][commander]) < 99:
            short.append(commander)
    elif commander in lists['test']:
        if len(lists['test'][commander]) < 99:
            short.append(commander)
    else:
        fail.append(commander)
len(short),len(fail)

(8, 0)

In [19]:
set([len(deck) for deck in lists['valid'].values()])

{52,
 70,
 83,
 96,
 97,
 99,
 109,
 144,
 145,
 152,
 157,
 161,
 162,
 164,
 168,
 170,
 185,
 218,
 226,
 229,
 232,
 233,
 235,
 239,
 241,
 244,
 248,
 249,
 250,
 253,
 257,
 258,
 259,
 261,
 262,
 263,
 264,
 266,
 267,
 269,
 270,
 272,
 273,
 275,
 277,
 279,
 282,
 283,
 284,
 285,
 286,
 287,
 288,
 290,
 291,
 292,
 294,
 298,
 305,
 307,
 308,
 310,
 313,
 314,
 315,
 317,
 318,
 319,
 329,
 359}

In [13]:
set([len(deck) for deck in baseline['valid'].values()])

{52, 70, 83, 96, 97, 99}

In [7]:
valid_list = lists['valid']
comm

'Kiki-Jiki, Mirror Breaker'

In [ ]:
cards = {card['sanitized']:{'total':card['num_decks'],comm:card['num_decks']}}

## synergy metric

In [15]:
for split in ['non_commander']:
    counter = 0
    for card in tqdm(lists[split]):
        if isinstance(lists[split][card],dict):
            continue
        counter+= 1
        if counter%200 == 0:
            print(f'pausing after request # {counter}')
            time.sleep(5)
        json_data = request_json(card, False)
        if json_data:
            lists[split][card] = {'total':json_data['container']['json_dict']['card']['num_decks'],
                                  'cards':lists[split][card]}

100%|██████████| 11794/11794 [00:00<00:00, 1182869.95it/s]


In [101]:
lists.keys()

dict_keys(['test', 'valid', 'non_commander'])

In [104]:
less_lists = {}
for split in lists:
    less_lists[split] = {}
    for commander in lists[split].keys():
        cards_less = {card['name']:card['num_decks'] for card in lists[split][commander]['cards']}
        less_lists[split][commander] = {'total':lists[split][commander]['total'],
                                        'cards':cards_less}

In [105]:
with open(os.path.join(DATA_PATH,'edhreclists.pkl'), 'wb') as f:
    pickle.dump(less_lists, f)

In [62]:
list(lists['non_commander'].values())[0]['cards']

[{'name': 'Arcane Signet',
  'sanitized': 'arcane-signet',
  'sanitized_wo': 'arcane-signet',
  'url': '/cards/arcane-signet',
  'synergy': 0.08,
  'num_decks': 2261244,
  'potential_decks': 2931462},
 {'name': 'Command Tower',
  'sanitized': 'command-tower',
  'sanitized_wo': 'command-tower',
  'url': '/cards/command-tower',
  'synergy': 0.03,
  'num_decks': 2211439,
  'potential_decks': 2949653},
 {'name': 'Exotic Orchard',
  'sanitized': 'exotic-orchard',
  'sanitized_wo': 'exotic-orchard',
  'url': '/cards/exotic-orchard',
  'synergy': 0.02,
  'num_decks': 1049343,
  'potential_decks': 2949920},
 {'name': 'Swords to Plowshares',
  'sanitized': 'swords-to-plowshares',
  'sanitized_wo': 'swords-to-plowshares',
  'url': '/cards/swords-to-plowshares',
  'synergy': 0.38,
  'num_decks': 893356,
  'potential_decks': 1340893},
 {'name': 'Swiftfoot Boots',
  'sanitized': 'swiftfoot-boots',
  'sanitized_wo': 'swiftfoot-boots',
  'url': '/cards/swiftfoot-boots',
  'synergy': 0.02,
  'num_deck

In [18]:
len([k for k,v in lists['non_commander'].items() if isinstance(v,dict) and len(v.keys())==2 and isinstance(v['cards'],list)])

11794

In [17]:
with open(os.path.join(DATA_PATH,'edhreclists.pkl'), 'wb') as f:
    pickle.dump(lists, f)

In [55]:
sol = request_json('Sol Ring')
sol['header'],sol['container']['json_dict']['card']['num_decks']

('Sol Ring (Card)', 2953387)

In [39]:
max([card['num_decks'] for card in lists['non_commander']['Sol Ring']])

2261244

In [26]:
lists['valid'][comm]['cards'][-1]

{'name': 'Roaming Throne',
 'sanitized': 'roaming-throne',
 'sanitized_wo': 'roaming-throne',
 'url': '/cards/roaming-throne',
 'synergy': -0.18,
 'num_decks': 11,
 'potential_decks': 157}

In [53]:
def synergy(a,b, lists, a_is_commander=False):
    """
    Calculate count-based synergy between two cards, according to EDHRec play data.
    If considering a commander, it should be the first card listed, and set a_is_commander to True.
    Return: a numerical score: log(ab_counts**2/(a_counts*b_counts))
    """
    # get a-card fequency
    a_set = False
    a_freq = 0
    a_cards = []
    if a_is_commander: # commander card
        for split in ['valid','test']:
            if a in lists[split]:
                a_freq = lists[split][a]['total']
                a_cards = lists[split][a]['cards']
                a_set = True
                break
    elif a in lists['non_commander']: # normal card we've already queried
        a_freq = lists['non_commander'][a]['total']
        a_cards = lists['non_commander'][a]['cards']
        a_set = True

    if not a_set: # need to query the API for a-card data
        a_data = None
        while a_data is None:
            a_data = request_json(a,a_is_commander)
        a_freq = a_data['container']['json_dict']['card']['num_decks']
        a_cards = a_data['cardlist']

    # get a and b co-fequency
    ab = [card for card in a_cards if card['name'] == b]
    if len(ab)>0:
        ab_freq = ab[0]['num_decks']
    else: # cards don't co-occur. assume 1 co-occurrence for logarithm
        ab_freq = 1

    # get b-card fequency
    b_set = False
    b_freq = 0
    if b in lists['non_commander']: # there can't be 2 commanders generally
        b_freq = lists['non_commander'][b]['total']
        b_set = True
    if not b_set: # need to query the API for a data
        b_data = None
        while b_data is None:
            b_data = request_json(b,False)
        b_freq = b_data['container']['json_dict']['card']['num_decks']

    ratio = ab_freq**2/(a_freq*b_freq) # squaring the numerator gives better spread
    return math.log(ratio)


In [54]:
comm = 'Stangg, Echo Warrior'
high_syn = 'Command Tower'
low_syn = 'Roaming Throne'

In [55]:
for a,b,c in [(high_syn, high_syn,False),(comm, high_syn,True),(comm, low_syn,True),(high_syn,low_syn,False)]:
    print(synergy(a,b, lists,a_is_commander=c))

-29.49589487285557
-7.713427787246376
-13.591501177382606
-4.306623108230701


In [56]:
math.mean([1,2,3])

AttributeError: module 'math' has no attribute 'mean'

# synergy metrics

In [4]:
from scrape_cardlists import request_json, CARDLISTS_PATH
lists = pickle.load(open(CARDLISTS_PATH, 'rb'))
DECK_PATH = os.path.abspath(os.path.join('..', 'data','decks'))

path='gpt'
deck_path = os.path.join(DECK_PATH, path)
print(deck_path)
deckfiles = os.listdir(deck_path)
deckfiles = [f for f in deckfiles if 'txt' in f]
len(deckfiles)

c:\Users\eviye\Documents\UCSD 2023-2024\dsc180ab\edhbuilder\data\decks\gpt


189

In [24]:
def request_json(name:str, is_commander, redirect=''):
    formatted_name = format_card_name(name)
    if redirect:
        print(f"Redirected to {redirect}")
        json_url = f"https://json.edhrec.com/pages{redirect}.json"
    else:
        json_url = f"https://json.edhrec.com/pages/{'commanders' if is_commander else 'cards'}/{formatted_name}.json"
    response = requests.get(json_url)
    if response.status_code == 200:
        json_data = response.json()
        if 'redirect' in json_data:
            return request_json(name, is_commander,redirect=json_data['redirect'])
        # print(f"JSON request successful!")
        return json_data
    else:
        print(f"JSON request for \"{name}\" ({formatted_name}) failed! Try different card name")

def synergy(a,b, lists, a_is_commander=False):
    """
    Calculate count-based synergy between two cards, according to EDHRec play data.
    If considering a commander, it should be the first card listed, and set a_is_commander to True.

    a: a card name
    b: a card name
    lists: a dictionary of cardlists, as returned by scrape_cardlists.py
    a_is_commander: a boolean indicating whether a is a commander card

    Return: a numerical score: log(ab_counts**2/(a_counts*b_counts))
    """
    # get a-card fequency
    a_set = False
    a_freq = 0
    a_cards = []
    if a_is_commander: # commander card
        for split in ['valid','test']:
            if a in lists[split]:
                a_freq = lists[split][a]['total']
                a_cards = lists[split][a]['cards']
                a_set = True
                break
    elif a in lists['non_commander']: # normal card we've already queried
        a_freq = lists['non_commander'][a]['total']
        a_cards = lists['non_commander'][a]['cards']
        a_set = True

    if not a_set: # need to query the API for a-card data
        a_data = None
        for _ in range(5):
            a_data = request_json(a,a_is_commander)
            if a_data is not None:
                break
        if a_data is None:
            raise ValueError('Failed to get data for card:',a)
        a_freq = a_data['container']['json_dict']['card']['num_decks']
        a_cards = {card['name']:card['num_decks'] for card in a_data['cardlist']}
        a_split = 'test' if a_is_commander else 'non_commander'
        lists[a_split][a] = {'cards':a_cards, 'total':a_freq} # save the data for later use
        print(f'Adding card to {a_split} list: {a}')

    # get a and b co-fequency
    if b in a_cards:
        ab_freq = a_cards[b]
    else: # cards don't co-occur. assume 1 co-occurrence for calculating logarithm
        ab_freq = 1

    # get b-card fequency
    b_set = False
    b_freq = 0
    if b in lists['non_commander']: # there can't be 2 commanders
        b_freq = lists['non_commander'][b]['total']
        b_set = True
    if not b_set: # need to query the API for a data
        b_data = None
        for _ in range(5):
            b_data = request_json(b,False)
            if b_data is not None:
                break
            print(repr(b))
        if b_data is None:
            raise ValueError('Failed to get data for card:',b)
        b_cards = {card['name']:card['num_decks'] for card in b_data['cardlist']}
        b_freq = b_data['container']['json_dict']['card']['num_decks']
        lists['non_commander'][b] = {'cards':b_cards, 'total':b_freq} # save the data for later use
        print(f'Adding card to non_commander list: {b}')

    ratio = ab_freq**2/(a_freq*b_freq) # squaring the numerator gives better spread
    return math.log(ratio), lists

def mean (l):
    return sum(l)/len(l)

In [25]:
commander = 'Ashad, the Lone Cyberman'
deck = ['Wizards of Thay', 'Mulldrifter', "Thieves' Tools",\
 'Counterspell', 'Wing Splicer', 'Through the Breach', 'Volo, Itinerant Scholar'\
, 'Torgaar, Famine Incarnate', 'Thunderheads', 'Tezzeret the Schemer', 'Whirlwin\
d Denial', 'Warbeast of Gorgoroth', 'Voidmage Apprentice', 'Vexing Scuttler', 'V\
odalian Mage', 'Whirler Rogue', 'Vandalblast', 'Tolarian Kraken', 'Tezzeret, Cru\
el Machinist', 'Time Elemental', 'Undead Alchemist', 'Storm-Kiln Artist', 'Wretc\
hed Gryff', 'Ugin, the Ineffable', 'The Eleventh Hour', 'Vex', 'Vizier of Many F\
aces', 'Turn // Burn', 'Mind Stone', 'Wandering Archaic // Explore the Vastlands\
', 'Viashino Heretic', 'Wrexial, the Risen Deep', 'Witness Protection', 'Volcani\
c Vision', 'Vedalken Blademaster', 'Vindictive Flamestoker', 'Zombie Boa', 'Voda\
lian Mystic', 'Vedalken Anatomist', 'Unexpected Windfall', 'Wandering Fumarole',\
 'Warping Wail', 'Warteye Witch', 'Unspeakable Symbol', 'Tombstalker', 'Teferi,\
Temporal Pilgrim', 'Talrand, Sky Summoner', 'Tar Pit Warrior', 'Wasteland Strang\
ler', 'Thousand-Year Storm', "Vraska's Contempt", 'Stone Idol Trap', 'Thraxodemo\
n', 'Virtue of Knowledge // Vantress Visions', 'Thieving Amalgam', 'Winding Cany\
ons', 'Vesuvan Shapeshifter', "Tamiyo's Logbook", 'Waste Away', 'Vicious Offerin\
g', "Veteran's Armaments", 'Teleportal', 'Violet Pall']


In [26]:
commander_sim = []
co_card_sim = []
print(f'evaluating deck: {commander}')
failed = set()
for i in tqdm(range(len(deck))):
    card = deck[i]
    if card in failed:
        continue
    try:
        sim, lists = synergy(commander,card,lists, a_is_commander=True)
        commander_sim.append(sim)
        card_sim = []
        for j in range(len(deck)):
            b = deck[j]
            print(card, b)
            if i == j or b in failed: # ignore synergy with itself, ignore failed cards
                continue
            try:
                sim, lists = synergy(card, b, lists)
                card_sim.append(sim)
            except ValueError as e: # b card not found
                print(e)
                # print(card, b)
                failed.add(b)
                continue
        co_card_sim.append(mean(card_sim))
    except ValueError as e:  # a card not found
        print(e)
        # print(card, b)
        failed.add(a)
        continue
return mean(commander_sim), mean(co_card_sim), lists

evaluating deck: Ashad, the Lone Cyberman


  0%|          | 0/63 [00:00<?, ?it/s]

Wizards of Thay Wizards of Thay
Wizards of Thay Mulldrifter
Wizards of Thay Thieves' Tools
Wizards of Thay Counterspell
Wizards of Thay Wing Splicer
Wizards of Thay Through the Breach
Wizards of Thay Volo, Itinerant Scholar
Wizards of Thay Torgaar, Famine Incarnate
Wizards of Thay Thunderheads
Wizards of Thay Tezzeret the Schemer
Wizards of Thay Whirlwind Denial
Wizards of Thay Warbeast of Gorgoroth
Wizards of Thay Voidmage Apprentice
Wizards of Thay Vexing Scuttler
Wizards of Thay Vodalian Mage
Wizards of Thay Whirler Rogue
Wizards of Thay Vandalblast
Wizards of Thay Tolarian Kraken
Wizards of Thay Tezzeret, Cruel Machinist
Wizards of Thay Time Elemental
Wizards of Thay Undead Alchemist
Wizards of Thay Storm-Kiln Artist
Wizards of Thay Wretched Gryff
Wizards of Thay Ugin, the Ineffable
Wizards of Thay The Eleventh Hour
Wizards of Thay Vex
Wizards of Thay Vizier of Many Faces
Wizards of Thay Turn // Burn


  0%|          | 0/63 [00:00<?, ?it/s]


UnboundLocalError: cannot access local variable 'json_data' where it is not associated with a value